# 🧠 量子ガイスターAI開発チュートリアル

このノートブックでは、PennyLaneとPyTorchを使った量子機械学習によるガイスターAIの開発方法を学びます。

## 📚 目次
1. [環境セットアップ](#1-環境セットアップ)
2. [基本的な量子回路](#2-基本的な量子回路)
3. [ガイスターゲームエンジン](#3-ガイスターゲームエンジン)
4. [量子AIアーキテクチャ](#4-量子aiアーキテクチャ)
5. [学習プロセス](#5-学習プロセス)
6. [性能評価と調整](#6-性能評価と調整)
7. [高度なカスタマイズ](#7-高度なカスタマイズ)

## 1. 環境セットアップ

必要なライブラリをインポートします。

In [1]:
# ===== WebUI設定ファイル読み込み =====
import json
import glob
import os
from datetime import datetime

def load_latest_config():
    """
    最新の設定ファイルを自動で読み込む
    """
    config_files = glob.glob("quantum_geister_config_*.json")
    if not config_files:
        print("❌ 設定ファイルが見つかりません。")
        print("💡 WebUIで設定を生成してから実行してください。")
        return None
    
    # 最新のファイルを取得
    latest_file = max(config_files, key=os.path.getctime)
    
    try:
        with open(latest_file, 'r', encoding='utf-8') as f:
            config = json.load(f)
        
        print(f"✅ 設定ファイル読み込み完了: {latest_file}")
        print(f"📅 生成日時: {config['learning_config']['timestamp']}")
        print(f"🧠 学習方法: {config['learning_config']['method']}")
        print(f"⚛️ 量子ビット数: {config['module_config']['quantum']['n_qubits']}")
        print(f"📚 レイヤー数: {config['module_config']['quantum']['n_layers']}")
        
        return config
    except Exception as e:
        print(f"❌ 設定ファイル読み込みエラー: {e}")
        return None

def create_default_config():
    """
    デフォルト設定を作成（設定ファイルが無い場合）
    """
    return {
        "learning_config": {
            "method": "reinforcement",
            "algorithm": "dqn",
            "timestamp": datetime.now().isoformat()
        },
        "module_config": {
            "quantum": {
                "n_qubits": 6,
                "n_layers": 2,
                "embedding_type": "angle",
                "entanglement": "linear",
                "total_params": 36
            },
            "reward": {
                "strategy": "balanced"
            },
            "qmap": {
                "method": "dqn",
                "state_dim": 252,
                "action_dim": 5
            },
            "action_selection": {
                "strategy": "epsilon"
            }
        },
        "hyperparameters": {
            "batch_size": 32,
            "epochs": 100,
            "learning_rate": 0.001,
            "optimizer": "adam"
        }
    }

# 設定ファイルを読み込み
config = load_latest_config()
if config is None:
    print("🔧 デフォルト設定を使用します")
    config = create_default_config()

# グローバル変数として設定
learning_config = config['learning_config']
module_config = config['module_config']
hyperparameters = config['hyperparameters']

print("\n" + "="*50)
print("🚀 WebUI連携設定完了")
print("="*50)

✅ 設定ファイル読み込み完了: quantum_geister_config_2025-09-23 (1).json
📅 生成日時: 2025-09-23T07:48:01.440Z
🧠 学習方法: reinforcement
⚛️ 量子ビット数: 4
📚 レイヤー数: 1

🚀 WebUI連携設定完了


In [3]:
import sys
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml
import matplotlib.pyplot as plt
import random
from collections import deque

# プロジェクトパスを追加
sys.path.append(str(Path.cwd()))
sys.path.append(str(Path.cwd() / "src"))

print("📦 ライブラリ読み込み完了")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"⚛️ PennyLane: {qml.__version__}")

ModuleNotFoundError: No module named 'numpy'

## 2. 基本的な量子回路

量子ガイスターAIの基盤となる量子回路を理解しましょう。

In [ ]:
# 6量子ビット量子デバイスを作成
n_qubits = 6
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def simple_quantum_circuit(inputs):
    """
    シンプルな量子回路の例
    
    Args:
        inputs: 入力パラメータ（6次元）
    
    Returns:
        量子状態の期待値
    """
    # 入力エンコーディング
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)
    
    # エンタングリング層
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    
    # 測定
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# テスト実行
test_input = np.random.rand(6) * np.pi
result = simple_quantum_circuit(test_input)
print(f"入力: {test_input[:3]:.3f}...")
print(f"出力: {result[:3]:.3f}...")

NameError: name 'qml' is not defined

## 3. ガイスターゲームエンジン

ガイスターゲームの基本的な操作を理解しましょう。

In [ ]:
from qugeister import GeisterEngine

# ゲームエンジンを初期化
game = GeisterEngine()
game.reset_game()

print("🎮 ガイスターゲーム初期化完了")
print(f"現在のプレイヤー: {game.current_player}")
print(f"ゲーム状態: {'終了' if game.game_over else '進行中'}")

# 合法手を取得
legal_moves = game.get_legal_moves('A')
print(f"プレイヤーAの合法手数: {len(legal_moves)}")
print(f"最初の3手: {legal_moves[:3]}")

# ボード状態の可視化
def visualize_board(game):
    """
    ゲームボードを簡単に可視化
    """
    board = game.get_board_state()
    print("\n📋 ボード状態 (6x6):")
    for row in board:
        print(' '.join([f'{cell:2.0f}' for cell in row]))

visualize_board(game)

ModuleNotFoundError: No module named 'qugeister'

# WebUI設定に基づく動的量子AIクラス
class WebUIQuantumAI(nn.Module):
    """
    WebUIの設定に基づいて動的に構築される量子AI
    
    設定ファイルからパラメータを読み込んで自動構築します。
    """
    
    def __init__(self, config):
        super().__init__()
        
        # 設定から量子パラメータを取得
        quantum_config = config['module_config']['quantum']
        self.n_qubits = quantum_config['n_qubits']
        self.n_layers = quantum_config['n_layers']
        self.embedding_type = quantum_config['embedding_type']
        self.entanglement = quantum_config['entanglement']
        
        # Q値マップ設定
        qmap_config = config['module_config']['qmap']
        self.action_dim = qmap_config['action_dim']
        
        print(f"🏗️ 量子AI構築中...")
        print(f"   量子ビット数: {self.n_qubits}")
        print(f"   レイヤー数: {self.n_layers}")
        print(f"   エンベディング: {self.embedding_type}")
        print(f"   エンタングルメント: {self.entanglement}")
        print(f"   行動次元: {self.action_dim}")
        
        # 量子デバイス
        self.dev = qml.device('default.qubit', wires=self.n_qubits)
        
        # 前処理層（CNN風）
        self.conv1 = nn.Conv2d(7, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # 量子回路パラメータ
        self.quantum_params = nn.Parameter(
            torch.randn(self.n_layers, self.n_qubits, 3) * 0.1
        )
        
        # 後処理層
        self.fc1 = nn.Linear(self.n_qubits, 64)
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, self.action_dim)
        
        print(f"✅ 量子AI構築完了 - パラメータ数: {sum(p.numel() for p in self.parameters())}")
    
    def quantum_circuit(self, inputs, params):
        """
        WebUI設定に基づく量子回路
        """
        # データエンコーディング（設定に基づく）
        for i in range(self.n_qubits):
            if self.embedding_type == 'angle':
                qml.RY(inputs[i], wires=i)
            elif self.embedding_type == 'amplitude':
                qml.RY(inputs[i] * np.pi / 2, wires=i)
                qml.RZ(inputs[i] * np.pi / 4, wires=i)
            else:  # iqp
                qml.RX(inputs[i], wires=i)
        
        # パラメータ化された層
        for layer in range(self.n_layers):
            # 回転ゲート
            for i in range(self.n_qubits):
                qml.RX(params[layer, i, 0], wires=i)
                qml.RY(params[layer, i, 1], wires=i)
                qml.RZ(params[layer, i, 2], wires=i)
            
            # エンタングルメント（設定に基づく）
            if self.entanglement == 'linear':
                for i in range(self.n_qubits - 1):
                    qml.CNOT(wires=[i, i + 1])
            elif self.entanglement == 'circular':
                for i in range(self.n_qubits - 1):
                    qml.CNOT(wires=[i, i + 1])
                qml.CNOT(wires=[self.n_qubits - 1, 0])  # 循環結合
            elif self.entanglement == 'full':
                for i in range(self.n_qubits):
                    for j in range(i+1, self.n_qubits):
                        qml.CZ(wires=[i, j])
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def forward(self, game_state):
        """
        フォワードパス
        """
        batch_size = game_state.size(0)
        
        # CNN処理
        x = torch.relu(self.conv1(game_state))
        x = torch.relu(self.conv2(x))
        x = self.global_pool(x).view(batch_size, -1)
        
        # 量子処理
        quantum_outputs = []
        for i in range(batch_size):
            quantum_input = torch.tanh(x[i]) * np.pi / 2
            
            # 量子回路をQNodeとして作成
            @qml.qnode(self.dev)
            def circuit(inputs, params):
                return self.quantum_circuit(inputs, params)
            
            q_out = circuit(quantum_input, self.quantum_params)
            quantum_outputs.append(torch.stack(q_out))
        
        quantum_tensor = torch.stack(quantum_outputs)
        
        # 後処理
        x = torch.relu(self.fc1(quantum_tensor))
        x = torch.relu(self.fc2(x))
        q_values = self.output(x)
        
        return q_values

# WebUI設定に基づいてAIを作成
webui_ai = WebUIQuantumAI(config)

# 設定内容の表示
print(f"\n📋 WebUI設定サマリー:")
print(f"   学習方法: {learning_config['method']}")
print(f"   アルゴリズム: {learning_config['algorithm']}")
print(f"   報酬戦略: {module_config['reward']['strategy']}")
print(f"   行動選択: {module_config['action_selection']['strategy']}")
print(f"   バッチサイズ: {hyperparameters['batch_size']}")
print(f"   学習率: {hyperparameters['learning_rate']}")
print(f"   オプティマイザ: {hyperparameters['optimizer']}")

In [ ]:
# 既存の量子AIクラスをインポート
from test_qubits_6 import QuantumBattleAI_6Qubits

class CustomQuantumAI(nn.Module):
    """
    カスタマイズ可能な量子AI
    
    このクラスでは学習の各部分を細かく制御できます。
    """
    
    def __init__(self, n_qubits=6, n_layers=2):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        
        # 量子デバイス
        self.dev = qml.device('default.qubit', wires=n_qubits)
        
        # 前処理層（CNN風）
        self.conv1 = nn.Conv2d(7, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # 量子回路パラメータ
        self.quantum_params = nn.Parameter(
            torch.randn(n_layers, n_qubits) * 0.1
        )
        
        # 後処理層
        self.fc1 = nn.Linear(n_qubits, 64)
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)  # Q値出力
        
    def quantum_circuit(self, inputs, params):
        """
        カスタマイズ可能な量子回路
        """
        # 入力エンコーディング
        for i in range(self.n_qubits):
            qml.RY(inputs[i], wires=i)
        
        # パラメータ化された層
        for layer in range(self.n_layers):
            # 回転ゲート
            for i in range(self.n_qubits):
                qml.RY(params[layer, i], wires=i)
            
            # エンタングリング
            for i in range(self.n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            qml.CNOT(wires=[self.n_qubits - 1, 0])  # 循環結合
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def forward(self, game_state):
        """
        フォワードパス
        """
        batch_size = game_state.size(0)
        
        # CNN処理
        x = torch.relu(self.conv1(game_state))
        x = torch.relu(self.conv2(x))
        x = self.global_pool(x).view(batch_size, -1)
        
        # 量子処理
        quantum_outputs = []
        for i in range(batch_size):
            quantum_input = torch.tanh(x[i]) * np.pi / 2
            
            # 量子回路をQNodeとして作成
            @qml.qnode(self.dev)
            def circuit(inputs, params):
                return self.quantum_circuit(inputs, params)
            
            q_out = circuit(quantum_input, self.quantum_params)
            quantum_outputs.append(torch.stack(q_out))
        
        quantum_tensor = torch.stack(quantum_outputs)
        
        # 後処理
        x = torch.relu(self.fc1(quantum_tensor))
        x = torch.relu(self.fc2(x))
        q_value = self.output(x)
        
        return q_value

# カスタムAIをテスト
custom_ai = CustomQuantumAI(n_qubits=6, n_layers=3)
print(f"🤖 カスタム量子AI作成完了")
print(f"パラメータ数: {sum(p.numel() for p in custom_ai.parameters())}")

def webui_training_loop(model, config, progress_callback=None):
    """
    WebUI設定に基づく学習ループ
    
    設定ファイルからハイパーパラメータを読み込んで自動学習
    """
    # ハイパーパラメータを設定から取得
    hyperparams = config['hyperparameters']
    batch_size = hyperparams['batch_size']
    episodes = hyperparams['epochs']
    learning_rate = hyperparams['learning_rate']
    optimizer_type = hyperparams['optimizer']
    
    # 報酬戦略を取得
    reward_strategy = config['module_config']['reward']['strategy']
    action_strategy = config['module_config']['action_selection']['strategy']
    
    print(f"🎯 WebUI設定による学習開始")
    print(f"   エピソード数: {episodes}")
    print(f"   バッチサイズ: {batch_size}")
    print(f"   学習率: {learning_rate}")
    print(f"   オプティマイザ: {optimizer_type}")
    print(f"   報酬戦略: {reward_strategy}")
    print(f"   行動戦略: {action_strategy}")
    
    # オプティマイザ設定
    if optimizer_type == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_type == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    else:  # rmsprop
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    
    loss_fn = nn.MSELoss()
    
    # 経験リプレイバッファー
    replay_buffer = deque(maxlen=10000)
    
    # 学習統計
    episode_rewards = []
    losses = []
    convergence_data = {
        'episodes': [],
        'rewards': [],
        'losses': [],
        'win_rate': []
    }
    
    # 報酬関数の設定
    def calculate_reward(game, player, reward_strategy):
        if game.game_over:
            if game.winner == player:
                if reward_strategy == 'escape':
                    return 150  # 脱出戦略では勝利報酬を高く
                elif reward_strategy == 'aggressive':
                    return 120  # 攻撃戦略
                else:
                    return 100  # 標準勝利報酬
            elif game.winner and game.winner != player:
                if reward_strategy == 'defensive':
                    return -80  # 守備戦略では敗北ペナルティを軽く
                else:
                    return -100
            else:
                return 0  # 引き分け
        else:
            if reward_strategy == 'balanced':
                return 2  # バランス戦略では継続報酬を高く
            else:
                return 1
    
    # ε-greedy設定
    epsilon_start = 0.9 if action_strategy == 'epsilon' else 0.0
    epsilon_end = 0.01
    epsilon_decay = 0.995
    
    for episode in range(episodes):
        # ゲーム初期化
        try:
            from qugeister import GeisterEngine
            game = GeisterEngine()
            game.reset_game()
        except ImportError:
            # ゲームエンジンが無い場合のダミー処理
            print("⚠️ ゲームエンジンが見つかりません。ダミー学習を実行します。")
            
            # ダミーデータで学習
            dummy_states = torch.randn(batch_size, 7, 6, 6)
            dummy_targets = torch.randn(batch_size, model.action_dim)
            
            optimizer.zero_grad()
            outputs = model(dummy_states)
            loss = loss_fn(outputs, dummy_targets)
            loss.backward()
            optimizer.step()
            
            episode_rewards.append(random.uniform(-10, 10))
            losses.append(loss.item())
            continue
        
        episode_reward = 0
        step_count = 0
        max_steps = 100
        
        while not game.game_over and step_count < max_steps:
            current_player = game.current_player
            legal_moves = game.get_legal_moves(current_player)
            
            if not legal_moves:
                break
            
            if current_player == 'A':  # AIの手番
                # 状態をエンコード
                state = game.get_board_state()
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
                
                # 7チャンネル形式に変換
                state_7ch = torch.zeros(1, 7, 6, 6)
                state_7ch[0, 0] = state_tensor[0]
                
                # 行動選択（設定に基づく）
                epsilon = max(epsilon_end, epsilon_start * (epsilon_decay ** episode))
                
                if action_strategy == 'epsilon' and random.random() < epsilon:
                    action = random.choice(legal_moves)
                elif action_strategy == 'boltzmann':
                    # Boltzmann探索（簡略版）
                    action = random.choice(legal_moves)
                else:  # greedy or ucb
                    action = random.choice(legal_moves)
                
                # 行動実行
                game.make_move(current_player, action)
                
                # 報酬計算
                reward = calculate_reward(game, current_player, reward_strategy)
                episode_reward += reward
                
                # 経験をバッファーに保存
                replay_buffer.append((state_7ch, action, reward))
                
            else:  # 相手（ランダム）の手番
                action = random.choice(legal_moves)
                game.make_move(current_player, action)
            
            step_count += 1
        
        episode_rewards.append(episode_reward)
        
        # バッチ学習
        if len(replay_buffer) >= batch_size and episode % 5 == 0:
            batch = random.sample(replay_buffer, batch_size)
            
            states = torch.cat([item[0] for item in batch])
            rewards = torch.FloatTensor([item[2] for item in batch])
            
            optimizer.zero_grad()
            q_values = model(states).mean(dim=1)  # 平均Q値
            loss = loss_fn(q_values, rewards)
            
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
        
        # 統計収集
        if episode % 20 == 0:
            recent_rewards = episode_rewards[-20:] if episode_rewards else [0]
            recent_losses = losses[-10:] if losses else [0]
            
            convergence_data['episodes'].append(episode)
            convergence_data['rewards'].append(np.mean(recent_rewards))
            convergence_data['losses'].append(np.mean(recent_losses) if recent_losses else 0)
            convergence_data['win_rate'].append(
                len([r for r in recent_rewards if r > 50]) / len(recent_rewards) * 100
            )
            
            print(f"Episode {episode}: 平均報酬={np.mean(recent_rewards):.1f}, "
                  f"損失={np.mean(recent_losses):.4f}, ε={epsilon:.3f}")
            
            # プログレスコールバック（WebUIとの連携用）
            if progress_callback:
                progress_callback({
                    'episode': episode,
                    'total_episodes': episodes,
                    'avg_reward': np.mean(recent_rewards),
                    'avg_loss': np.mean(recent_losses) if recent_losses else 0,
                    'epsilon': epsilon,
                    'win_rate': convergence_data['win_rate'][-1]
                })
    
    print(f"✅ 学習完了: 最終平均報酬 = {np.mean(episode_rewards[-20:]):.2f}")
    
    return {
        'rewards': episode_rewards,
        'losses': losses,
        'convergence_data': convergence_data,
        'final_performance': {
            'avg_reward': np.mean(episode_rewards[-20:]) if episode_rewards else 0,
            'total_episodes': episodes,
            'final_win_rate': convergence_data['win_rate'][-1] if convergence_data['win_rate'] else 0
        }
    }

# WebUI設定による学習実行
print("🧪 WebUI設定学習テスト開始...")
results = webui_training_loop(webui_ai, config)

# 結果可視化
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(results['rewards'])
plt.title('エピソード報酬 (WebUI設定)')
plt.xlabel('エピソード')
plt.ylabel('報酬')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
if results['losses']:
    plt.plot(results['losses'])
    plt.title('学習損失')
    plt.xlabel('更新ステップ')
    plt.ylabel('損失')
    plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
conv_data = results['convergence_data']
if conv_data['episodes']:
    plt.plot(conv_data['episodes'], conv_data['win_rate'], 'g-', label='勝率')
    plt.plot(conv_data['episodes'], conv_data['rewards'], 'b-', label='平均報酬')
    plt.title('収束分析')
    plt.xlabel('エピソード')
    plt.ylabel('値')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 最終結果表示
final_perf = results['final_performance']
print(f"\n📊 最終性能結果:")
print(f"   平均報酬: {final_perf['avg_reward']:.2f}")
print(f"   勝率: {final_perf['final_win_rate']:.1f}%")
print(f"   総エピソード: {final_perf['total_episodes']}")

# 設定保存（学習結果付き）
config['training_results'] = results['final_performance']
with open(f"quantum_training_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json", 'w') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print("💾 学習結果を保存しました")

In [ ]:
def custom_training_loop(model, episodes=100, batch_size=8):
    """
    カスタマイズ可能な学習ループ
    
    この関数では学習の各ステップを詳細に制御できます。
    """
    # 学習設定
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    
    # 経験リプレイバッファー
    replay_buffer = deque(maxlen=1000)
    
    # 学習統計
    episode_rewards = []
    losses = []
    
    print(f"🎯 学習開始: {episodes}エピソード")
    
    for episode in range(episodes):
        # ゲーム初期化
        game = GeisterEngine()
        game.reset_game()
        
        episode_reward = 0
        step_count = 0
        max_steps = 50  # エピソード長制限
        
        while not game.game_over and step_count < max_steps:
            current_player = game.current_player
            legal_moves = game.get_legal_moves(current_player)
            
            if not legal_moves:
                break
            
            if current_player == 'A':  # AIの手番
                # 状態をエンコード
                state = game.get_board_state()
                state_tensor = torch.FloatTensor(state).unsqueeze(0)  # チャンネル次元追加
                
                # 7チャンネル形式に変換（簡略化）
                state_7ch = torch.zeros(1, 7, 6, 6)
                state_7ch[0, 0] = state_tensor[0]  # 基本ボード情報
                
                # ε-greedy行動選択
                epsilon = max(0.01, 0.5 * (0.995 ** episode))
                
                if random.random() < epsilon:
                    # ランダム行動
                    action = random.choice(legal_moves)
                else:
                    # AIによる行動選択（簡略化）
                    action = random.choice(legal_moves)
                
                # 行動実行
                game.make_move(current_player, action)
                
                # 報酬計算
                if game.game_over:
                    if game.winner == 'A':
                        reward = 100  # 勝利
                    elif game.winner == 'B':
                        reward = -100  # 敗北
                    else:
                        reward = 0  # 引き分け
                else:
                    reward = 1  # 継続報酬
                
                episode_reward += reward
                
                # 経験をバッファーに保存
                replay_buffer.append((state_7ch, action, reward))
                
            else:  # 相手（ランダム）の手番
                action = random.choice(legal_moves)
                game.make_move(current_player, action)
            
            step_count += 1
        
        episode_rewards.append(episode_reward)
        
        # バッチ学習
        if len(replay_buffer) >= batch_size and episode % 10 == 0:
            # ランダムサンプリング
            batch = random.sample(replay_buffer, batch_size)
            
            # バッチデータを準備
            states = torch.cat([item[0] for item in batch])
            rewards = torch.FloatTensor([item[2] for item in batch])
            
            # フォワードパス
            optimizer.zero_grad()
            q_values = model(states).squeeze()
            
            # 損失計算
            loss = loss_fn(q_values, rewards)
            losses.append(loss.item())
            
            # バックプロパゲーション
            loss.backward()
            optimizer.step()
        
        # 進捗表示
        if episode % 20 == 0:
            avg_reward = np.mean(episode_rewards[-20:]) if episode_rewards else 0
            avg_loss = np.mean(losses[-10:]) if losses else 0
            print(f"Episode {episode}: 平均報酬={avg_reward:.1f}, 損失={avg_loss:.4f}")
    
    return episode_rewards, losses

# 短期学習テスト
print("🧪 短期学習テスト開始...")
rewards, losses = custom_training_loop(custom_ai, episodes=50, batch_size=4)

# 結果可視化
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(rewards)
plt.title('エピソード報酬')
plt.xlabel('エピソード')
plt.ylabel('報酬')

plt.subplot(1, 2, 2)
if losses:
    plt.plot(losses)
    plt.title('学習損失')
    plt.xlabel('更新ステップ')
    plt.ylabel('損失')

plt.tight_layout()
plt.show()

print(f"✅ 学習完了: 平均報酬 = {np.mean(rewards):.2f}")

## 6. 性能評価と調整

学習済みモデルの性能を評価し、ハイパーパラメータを調整します。

In [ ]:
def evaluate_model(model, n_games=10):
    """
    モデルの性能を評価
    """
    model.eval()
    wins = 0
    draws = 0
    losses = 0
    
    with torch.no_grad():
        for game_idx in range(n_games):
            game = GeisterEngine()
            game.reset_game()
            
            step_count = 0
            max_steps = 50
            
            while not game.game_over and step_count < max_steps:
                current_player = game.current_player
                legal_moves = game.get_legal_moves(current_player)
                
                if not legal_moves:
                    break
                
                # 両プレイヤーともランダム（デモ用）
                action = random.choice(legal_moves)
                game.make_move(current_player, action)
                step_count += 1
            
            # 結果集計
            if game.winner == 'A':
                wins += 1
            elif game.winner == 'B':
                losses += 1
            else:
                draws += 1
    
    model.train()
    
    return {
        'wins': wins,
        'draws': draws, 
        'losses': losses,
        'win_rate': wins / n_games * 100
    }

# 性能評価
results = evaluate_model(custom_ai, n_games=20)
print("📊 性能評価結果:")
print(f"勝利: {results['wins']}/20 ({results['win_rate']:.1f}%)")
print(f"引分: {results['draws']}/20")
print(f"敗北: {results['losses']}/20")

# ハイパーパラメータ調整のヒント
print("\n🔧 ハイパーパラメータ調整のヒント:")
print("1. 学習率: 0.001-0.01 (高すぎると不安定、低すぎると遅い)")
print("2. バッチサイズ: 8-32 (メモリとの兼ね合い)")
print("3. エピソード数: 1000-5000 (十分な学習のため)")
print("4. ε値: 0.1-0.5 開始、0.01まで減衰 (探索と活用のバランス)")
print("5. 量子回路層数: 2-5層 (表現力と計算コストのトレードオフ)")

## 7. 高度なカスタマイズ

より高度な量子回路設計や学習手法を試してみましょう。

In [ ]:
# 高度な量子回路設計
class AdvancedQuantumCircuit:
    """
    より複雑な量子回路設計
    """
    
    def __init__(self, n_qubits=6):
        self.n_qubits = n_qubits
        self.dev = qml.device('default.qubit', wires=n_qubits)
    
    def amplitude_encoding_circuit(self, data, params):
        """
        振幅エンコーディングを使用した回路
        """
        # 正規化
        norm = torch.norm(data)
        if norm > 0:
            data = data / norm
        
        # 振幅エンコーディング（簡略版）
        for i in range(self.n_qubits):
            qml.RY(data[i] * np.pi, wires=i)
        
        # Strongly Entangling Layers
        qml.StronglyEntanglingLayers(params, wires=range(self.n_qubits))
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def variational_circuit(self, inputs, params):
        """
        変分量子回路
        """
        # データエンコーディング
        for i in range(self.n_qubits):
            qml.Hadamard(wires=i)
            qml.RZ(inputs[i], wires=i)
        
        # 変分層
        for layer in range(len(params)):
            # 回転ゲート
            for i in range(self.n_qubits):
                qml.RX(params[layer][i][0], wires=i)
                qml.RY(params[layer][i][1], wires=i)
                qml.RZ(params[layer][i][2], wires=i)
            
            # エンタングリング
            for i in range(self.n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            if self.n_qubits > 2:
                qml.CNOT(wires=[self.n_qubits - 1, 0])
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]

# 高度な学習手法
def parameter_shift_training(model, data, target, lr=0.01):
    """
    パラメータシフト法による勾配計算
    
    量子回路特有の勾配計算手法
    """
    gradients = []
    
    for param in model.parameters():
        if param.requires_grad:
            param_grad = torch.zeros_like(param)
            
            for i in range(param.numel()):
                # パラメータを+π/2シフト
                param.data.view(-1)[i] += np.pi / 2
                output_plus = model(data)
                
                # パラメータを-π/2シフト
                param.data.view(-1)[i] -= np.pi
                output_minus = model(data)
                
                # 元に戻す
                param.data.view(-1)[i] += np.pi / 2
                
                # 勾配計算
                grad = (output_plus - output_minus) / 2
                param_grad.view(-1)[i] = grad.item()
            
            gradients.append(param_grad)
    
    return gradients

# 量子回路の可視化
def visualize_quantum_circuit():
    """
    量子回路の構造を可視化
    """
    n_qubits = 4  # 可視化用に小さくする
    dev = qml.device('default.qubit', wires=n_qubits)
    
    @qml.qnode(dev)
    def demo_circuit(params):
        # データエンコーディング
        for i in range(n_qubits):
            qml.RY(params[i], wires=i)
        
        # エンタングリング
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        
        return qml.expval(qml.PauliZ(0))
    
    # 回路図を生成
    params = np.random.rand(n_qubits)
    print("🔧 量子回路の構造:")
    print(qml.draw(demo_circuit)(params))

visualize_quantum_circuit()

print("\n🎓 学習のポイント:")
print("1. 量子回路の深さ: 浅い回路から始めて徐々に複雑にする")
print("2. エンコーディング方法: Angle, Amplitude, IQPなど様々な手法を試す")
print("3. 測定方法: PauliZ, PauliX, PauliYの組み合わせで情報を抽出")
print("4. バリエーショナル層: 表現力と学習可能性のバランス")
print("5. ノイズ耐性: 実機では量子ノイズを考慮した設計が重要")

In [ ]:
# ===== WebUIとの双方向連携機能 =====

class WebUIBridge:
    """
    WebUIとJupyter Notebookの橋渡しクラス
    """
    
    def __init__(self):
        self.status_file = "jupyter_status.json"
        self.results_file = "jupyter_results.json"
    
    def send_progress_to_webui(self, progress_data):
        """
        学習進捗をWebUIに送信
        """
        progress_data['timestamp'] = datetime.now().isoformat()
        progress_data['status'] = 'training'
        
        try:
            with open(self.status_file, 'w') as f:
                json.dump(progress_data, f, indent=2)
            return True
        except Exception as e:
            print(f"⚠️ 進捗送信エラー: {e}")
            return False
    
    def send_results_to_webui(self, results, config):
        """
        学習結果をWebUIに送信
        """
        webui_results = {
            'timestamp': datetime.now().isoformat(),
            'status': 'completed',
            'config_used': config,
            'performance': results['final_performance'],
            'training_data': {
                'total_episodes': len(results['rewards']),
                'final_avg_reward': results['final_performance']['avg_reward'],
                'convergence_achieved': results['final_performance']['final_win_rate'] > 60,
                'loss_trend': 'decreasing' if len(results['losses']) > 10 and 
                             results['losses'][-1] < results['losses'][5] else 'stable'
            },
            'recommendations': self._generate_recommendations(results)
        }
        
        try:
            with open(self.results_file, 'w') as f:
                json.dump(webui_results, f, indent=2, ensure_ascii=False)
            print(f"✅ 結果をWebUIに送信: {self.results_file}")
            return True
        except Exception as e:
            print(f"⚠️ 結果送信エラー: {e}")
            return False
    
    def _generate_recommendations(self, results):
        """
        学習結果に基づく推奨設定を生成
        """
        recommendations = []
        
        avg_reward = results['final_performance']['avg_reward']
        win_rate = results['final_performance']['final_win_rate']
        
        if avg_reward < 20:
            recommendations.append({
                'type': 'learning_rate',
                'message': '学習率を上げることを推奨します (0.01-0.05)',
                'priority': 'high'
            })
        
        if win_rate < 40:
            recommendations.append({
                'type': 'exploration',
                'message': 'ε値を調整して探索を増やすことを推奨します',
                'priority': 'medium'
            })
        
        if len(results['losses']) > 0 and results['losses'][-1] > results['losses'][0]:
            recommendations.append({
                'type': 'overfitting',
                'message': '過学習の可能性があります。正則化を強化してください',
                'priority': 'high'
            })
        
        return recommendations

# WebUI連携インスタンス作成
webui_bridge = WebUIBridge()

# 進捗コールバック関数
def progress_callback(progress_data):
    """
    学習進捗をWebUIに送信するコールバック
    """
    webui_bridge.send_progress_to_webui(progress_data)
    
    # Jupyter内でも表示
    print(f"📊 Progress: Episode {progress_data['episode']}/{progress_data['total_episodes']} "
          f"- Reward: {progress_data['avg_reward']:.1f}, "
          f"Win Rate: {progress_data['win_rate']:.1f}%")

print("🌉 WebUI連携ブリッジ準備完了")

# ===== 高度な学習とモニタリング =====

def advanced_training_with_monitoring(model, config):
    """
    高度な学習監視機能付きトレーニング
    """
    print("🚀 高度な学習・監視システム開始")
    
    # 学習前の初期状態をWebUIに送信
    initial_status = {
        'episode': 0,
        'total_episodes': config['hyperparameters']['epochs'],
        'avg_reward': 0.0,
        'avg_loss': 0.0,
        'epsilon': 0.9,
        'win_rate': 0.0,
        'phase': 'initialization'
    }
    webui_bridge.send_progress_to_webui(initial_status)
    
    # 学習実行（プログレスコールバック付き）
    results = webui_training_loop(model, config, progress_callback)
    
    # 最終結果をWebUIに送信
    webui_bridge.send_results_to_webui(results, config)
    
    return results

# 学習実行とモニタリング
print("🎯 高度な学習システムでWebUI連携テスト...")
advanced_results = advanced_training_with_monitoring(webui_ai, config)

# 詳細分析と可視化
plt.figure(figsize=(20, 10))

# 1. 学習曲線
plt.subplot(2, 4, 1)
plt.plot(advanced_results['rewards'])
plt.title('Learning Curve')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.grid(True, alpha=0.3)

# 2. 損失曲線
plt.subplot(2, 4, 2)
if advanced_results['losses']:
    plt.plot(advanced_results['losses'])
    plt.title('Loss Curve')
    plt.xlabel('Training Step')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)

# 3. 移動平均報酬
plt.subplot(2, 4, 3)
if len(advanced_results['rewards']) > 20:
    moving_avg = np.convolve(advanced_results['rewards'], np.ones(20)/20, mode='valid')
    plt.plot(moving_avg)
    plt.title('Moving Average Reward')
    plt.xlabel('Episode')
    plt.ylabel('Avg Reward (20 episodes)')
    plt.grid(True, alpha=0.3)

# 4. 勝率推移
plt.subplot(2, 4, 4)
conv_data = advanced_results['convergence_data']
if conv_data['episodes']:
    plt.plot(conv_data['episodes'], conv_data['win_rate'])
    plt.title('Win Rate')
    plt.xlabel('Episode')
    plt.ylabel('Win Rate (%)')
    plt.grid(True, alpha=0.3)

# 5. 報酬分布
plt.subplot(2, 4, 5)
plt.hist(advanced_results['rewards'], bins=30, alpha=0.7)
plt.title('Reward Distribution')
plt.xlabel('Reward')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 6. 学習率vs性能
plt.subplot(2, 4, 6)
episodes = range(len(advanced_results['rewards']))
rewards = advanced_results['rewards']
if episodes and rewards:
    # 学習段階別の性能
    early = np.mean(rewards[:len(rewards)//3]) if len(rewards) > 3 else 0
    mid = np.mean(rewards[len(rewards)//3:2*len(rewards)//3]) if len(rewards) > 3 else 0
    late = np.mean(rewards[2*len(rewards)//3:]) if len(rewards) > 3 else 0
    
    plt.bar(['Early', 'Mid', 'Late'], [early, mid, late])
    plt.title('Performance by Phase')
    plt.ylabel('Average Reward')
    plt.grid(True, alpha=0.3)

# 7. 収束分析
plt.subplot(2, 4, 7)
if conv_data['losses']:
    plt.plot(conv_data['episodes'], conv_data['losses'], 'r-', label='Loss')
    plt.plot(conv_data['episodes'], conv_data['rewards'], 'b-', label='Reward')
    plt.title('Convergence Analysis')
    plt.xlabel('Episode')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True, alpha=0.3)

# 8. 最終統計
plt.subplot(2, 4, 8)
final_stats = [
    advanced_results['final_performance']['avg_reward'],
    advanced_results['final_performance']['final_win_rate'],
    len(advanced_results['rewards']),
    len(advanced_results['losses'])
]
labels = ['Avg Reward', 'Win Rate', 'Episodes', 'Updates']
plt.bar(labels, final_stats)
plt.title('Final Statistics')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🎊 WebUI連携学習完了!")
print(f"📈 最終性能: 平均報酬 {advanced_results['final_performance']['avg_reward']:.2f}")
print(f"🏆 勝率: {advanced_results['final_performance']['final_win_rate']:.1f}%")
print(f"💾 結果ファイル: {webui_bridge.results_file}")
print(f"📊 ステータスファイル: {webui_bridge.status_file}")

# 推奨設定の表示
if 'training_data' in advanced_results:
    print(f"\n💡 学習状況診断:")
    print(f"   収束状況: {'✅ 良好' if advanced_results['final_performance']['final_win_rate'] > 60 else '⚠️ 要改善'}")
    print(f"   学習安定性: {'✅ 安定' if len(advanced_results['losses']) > 0 else '⚠️ 不安定'}")
    print(f"   推奨次回実行: {'長期学習' if advanced_results['final_performance']['avg_reward'] > 30 else '設定調整'}")

## 🔄 WebUIとの双方向連携

WebUIで設定した内容がJupyter Notebookに自動反映され、学習結果をWebUIに送り返すことができます。

## 🎯 実践的な使い方

### 基本的な学習フロー:

1. **データ準備**: ゲーム状態を7チャンネル形式で表現
2. **回路設計**: 問題に応じて量子回路を設計
3. **学習実行**: DQNまたはカスタム学習ループで訓練
4. **評価**: 対戦テストで性能を確認
5. **調整**: ハイパーパラメータを最適化

### パラメータ調整の指針:

- **学習が不安定**: 学習率を下げる、バッチサイズを小さくする
- **学習が遅い**: 学習率を上げる、エピソード数を増やす
- **過学習**: 正則化を強める、回路を浅くする
- **汎化性能が低い**: データ拡張、ドロップアウト追加

### 次のステップ:

1. 実際の量子デバイス（IBM Quantum、Rigetti）での実行
2. より複雑なゲーム環境での評価
3. 他の量子機械学習アルゴリズムとの比較
4. ハイブリッド古典-量子アーキテクチャの探索